In [1]:
%cd /content/drive/MyDrive/nlp

/content/drive/MyDrive/nlp


In [2]:
from collections import defaultdict as dd
import numpy as np

In [3]:
bigrams = dd(lambda : dd(int))
r_bigrams = dd(lambda : dd(int))
file = open('2grams.txt', 'r')

for e in file:
    if e[-1] == '\n':
        e = e[:-1]
    e = e.lower()
    l, first, second = e.split(' ')
    bigrams[first][second] += int(l)
    r_bigrams[second][first] += int(l)
    if int(l) <= 1:
        break

file.close()
sum_of_bigrams = 0
for e in bigrams:
    for f in bigrams[e]:
        sum_of_bigrams += bigrams[e][f]

file = open('unigrams.txt', 'r')
unigrams = dict()
for e in file:
    if e[-1] == '\n':
        e = e[:-1]
    e = e.lower()
    first, second = e.split(' ')
    unigrams[first] = int(second)

file.close()
sum_of_unigrams = sum(unigrams[e] for e in unigrams)

In [4]:
file = open('supertags.txt', 'r')
tags = dict()
for i in file:
    e, f = i.split(' ')
    e = e.lower()
    if f[-1] == '\n':
        f = f[:-1]
    tags[e] = f

file.close()

def tag(s):
    if s in tags:
        return tags[s]
    else:
        return ('^' + s)[-3:]


tbigrams = dd(lambda : dd(int))
tunigrams = dd(int)
for e in bigrams:
    for f in bigrams[e]:
        tbigrams[tag(e)][tag(f)] += bigrams[e][f]
for e in unigrams:
    tunigrams[tag(e)] += unigrams[e]

In [5]:
print(sum_of_bigrams)
print(sum_of_unigrams)

434398049
451846640


$$PMI(w_1,w_2) = \log\frac{P(w_1,w_2)}{P(w_1)\cdot P(w_2)}$$
$$PPMI(w_1, w_2) = \max(PMI(w_1,w_2), 0)$$

In [6]:
def P_big(w1, w2):
    return np.log(bigrams[w1][w2]/sum_of_bigrams + 1e-10)

def P_uni(w1):
    return np.log(unigrams[w1]/sum_of_unigrams + 1e-10)

def PMI(w1,w2):
    return P_big(w1, w2) - (P_uni(w1) + P_uni(w2))

In [7]:
def PPMI(w1, w2):
    return max(PMI(w1,w2), 0)

$$PSM(w_1,w_2) = f(w_1,w_2) \cdot \left(\log f(w_1,w_2) - \log \frac{f(w_1)\cdot f(w_2)}{\sum_i f(w_i)} - 1 \right)$$
gdzie $f$ to statystyki bigramowe/unigramowe.

In [8]:
def PSM(w1, w2):
    def f0(w1, w2):
        return unigrams[w1]*unigrams[w2]/sum_of_unigrams
    f = bigrams[w1][w2]
    return f * (np.log(f + 1e-10) - np.log(f0(w1,w2) + 1e-10) - 1 )

$$tagPMI = PMI(w_1,w_2) + PMI(t_1,t_2)$$
gdzie $t_1, t_2$ to tagi słów $w_1,w_2$.

In [9]:
def P_big_t(t1, t2):
    return np.log(tbigrams[t1][t2]/sum_of_bigrams + 1e-10)

def P_uni_t(t1):
    return np.log(tunigrams[t1]/sum_of_unigrams + 1e-10)

def PMI_t(w1,w2):
    t1 = tag(w1)
    t2 = tag(w2)
    return P_big_t(t1, t2) - (P_uni_t(t1) + P_uni_t(t2))


def PMI_WandT(w1, w2):
    return PMI(w1,w2) + PMI_t(w1,w2)


In [10]:
def top_k(w, f, k):
    bests = [(0, '')] * k
    for e in bigrams[w]:
        n = f(w,e)
        if n >= bests[k-1][0]:
            bests = [(n, e)] + bests[:-1]
            bests.sort(key=lambda x: x[0], reverse=True)
    bestsr = [(0, '')] * k
    for e in r_bigrams[w]:
        if e == '<bos>':
            continue
        n = f(e,w)
        if n >= bestsr[k-1][0]:
            bestsr = [(n, e)] + bestsr[:-1]
            bestsr.sort(key=lambda x: x[0], reverse=True)
        
    return bests, bestsr

In [52]:
word = 'pies'
print(word)
print('{:^70} {:^70} {:^80}'.format('PPMI', 'PSM', 'tPMI'))
print('{:40} {:40} {:40} {:40} {:40} {:30}'.format('Następniki', 'Poprzedniki', 'Następniki', 'Poprzedniki', 'Następniki', 'Poprzedniki'))
e1, f1 = top_k(word, PPMI, 10)
e2, f2 = top_k(word, PSM, 10)
e3, f3 = top_k(word, PMI_WandT, 10)
for i in range(len(e)):
    print('{:03.5f} {:30} {:03.5f} {:30} {:03.5f} {:30} {:03.5f} {:30} {:03.5f} {:30} {:03.5f} {}'.format(
            e1[i][0], e1[i][1], f1[i][0], f1[i][1], e2[i][0], e2[i][1], f2[i][0], f2[i][1], e3[i][0], e3[i][1], f3[i][0], f3[i][1]))

pies
                                 PPMI                                                                   PSM                                                                         tPMI                                      
Następniki                               Poprzedniki                              Następniki                               Poprzedniki                              Następniki                               Poprzedniki                   
11.30188 descalzos                      11.30124 bernenski                      668.53500 pogrzebany                     511.48712 mój                            15.93337 descalzos                      12.65893 dorosły
11.30124 stróżująco-obronny             11.30124 oczipowany                     443.57206 pasterski                      486.56526 twój                           13.67837 chodzki                        12.39362 berneński
11.08076 chodzki                        11.08076 ujadający                      374.18877 myśliw

In [48]:
word = 'kot'
print(word)
print('{:^70} {:^70} {:^80}'.format('PPMI', 'PSM', 'tPMI'))
print('{:40} {:40} {:40} {:40} {:40} {:30}'.format('Następniki', 'Poprzedniki', 'Następniki', 'Poprzedniki', 'Następniki', 'Poprzedniki'))
e1, f1 = top_k(word, PPMI, 10)
e2, f2 = top_k(word, PSM, 10)
e3, f3 = top_k(word, PMI_WandT, 10)
for i in range(len(e)):
    print('{:.5f} {:30} {:.5f} {:30} {:.5f} {:30} {:.5f} {:30} {:.5f} {:30} {:.5f} {}'.format(
            e1[i][0], e1[i][1], f1[i][0], f1[i][1], e2[i][0], e2[i][1], f2[i][0], f2[i][1], e3[i][0], e3[i][1], f3[i][0], f3[i][1]))

kot
                                 PPMI                                                                   PSM                                                                         tPMI                                      
Następniki                               Poprzedniki                              Następniki                               Poprzedniki                              Następniki                               Poprzedniki                   
11.84886 naplakal                       11.84928 tirin                          524.04692 napłakał                       1177.85359 maciej                         15.54564 meowth                         15.03463 tirin
11.73427 napłakał                       11.16682 gadajacy                       170.34014 pospolity                      571.31415 czarny                         14.78221 czarnołapy                     14.35379 jewhenij
11.45079 burmański                      10.76509 jewhenij                       123.71421 domowy   

In [50]:
word = 'chłopiec'
print(word)
print('{:^70} {:^70} {:^80}'.format('PPMI', 'PSM', 'tPMI'))
print('{:40} {:40} {:40} {:40} {:40} {:30}'.format('Następniki', 'Poprzedniki', 'Następniki', 'Poprzedniki', 'Następniki', 'Poprzedniki'))
e1, f1 = top_k(word, PPMI, 10)
e2, f2 = top_k(word, PSM, 10)
e3, f3 = top_k(word, PMI_WandT, 10)
for i in range(len(e)):
    print('{:.5f} {:30} {:.5f} {:30} {:.5f} {:30} {:.5f} {:30} {:.5f} {:30} {:.5f} {}'.format(
            e1[i][0], e1[i][1], f1[i][0], f1[i][1], e2[i][0], e2[i][1], f2[i][0], f2[i][1], e3[i][0], e3[i][1], f3[i][0], f3[i][1]))

chłopiec
                                 PPMI                                                                   PSM                                                                         tPMI                                      
Następniki                               Poprzedniki                              Następniki                               Poprzedniki                              Następniki                               Poprzedniki                   
10.90248 obierający                     11.80542 bestia-zielony                 178.10998 miał                           902.07326 mały                           12.12077 obierający                     13.23603 11-letni
10.21382 popatrzał                      10.71703 anaruk                         147.87197 został                         785.42789 młody                          11.35954 popatrzał                      13.02770 14-letni
9.61860 zapalający                     10.42265 trzynastoletni                 132.95439 był

In [54]:
word = 'dziewczynka'
print(word)
print('{:^70} {:^70} {:^80}'.format('PPMI', 'PSM', 'tPMI'))
print('{:40} {:40} {:40} {:40} {:40} {:30}'.format('Następniki', 'Poprzedniki', 'Następniki', 'Poprzedniki', 'Następniki', 'Poprzedniki'))
e1, f1 = top_k(word, PPMI, 10)
e2, f2 = top_k(word, PSM, 10)
e3, f3 = top_k(word, PMI_WandT, 10)
for i in range(len(e)):
    print('{:03.5f} {:30} {:03.5f} {:30} {:03.5f} {:30} {:03.5f} {:30} {:03.5f} {:30} {:03.5f} {}'.format(
            e1[i][0], e1[i][1], f1[i][0], f1[i][1], e2[i][0], e2[i][1], f2[i][0], f2[i][1], e3[i][0], e3[i][1], f3[i][0], f3[i][1]))

dziewczynka
                                 PPMI                                                                   PSM                                                                         tPMI                                      
Następniki                               Poprzedniki                              Następniki                               Poprzedniki                              Następniki                               Poprzedniki                   
11.48803 zzieleniała                    12.17007 3,5-letnia                     101.17083 miała                          1433.50826 mała                           13.87168 bojąca                         13.47863 dwunastoletnia
10.97682 bojąca                         11.77200 1,5-roczna                     93.71574 została                        173.55058 letnia                         13.18215 zzieleniała                    13.26526 ośmioletnia
9.72728 zafascynuje                    10.56720 7-letnia                       7